# Исследование объявлений о продаже квартир

В вашем распоряжении данные сервиса Яндекс Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах за несколько лет. Вам нужно научиться определять рыночную стоимость объектов недвижимости. Для этого проведите исследовательский анализ данных и установите параметры, влияющие на цену объектов. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность.

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта и других объектов — эти данные автоматически получены из геосервисов. Количество парков и водоёмов также заполняется без участия пользователя.

## Откройте файл с данными и изучите общую информацию

In [84]:
import pandas as pd
pd.set_option('display.max_columns', None) # служебная строка для печати всех столбцов таблицы на экран
import matplotlib.pyplot as plt
import re
import numpy as np
import seaborn as sns
warnings.simplefilter("ignore")
pd.set_option('display.float_format', '{:.2f}'.format)

NameError: name 'warnings' is not defined

In [ ]:
try:
    df = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')
except:
    df = pd.read_csv('C:\\python\\real_estate_data.csv', sep='\t')

**Выведем первые 20 строчек датафрейма**

In [ ]:
df.head(20)

Вводим переменную old_size для того, чтобы после удаления пропусков и аномалий посчитать отношение нового размера датафрейма к старому.

In [ ]:
old_size = df.shape[0]  # сохраняем размер датафрейма до изменений

**Выведем основную информацию о датафрейме с помощью метода info()**

In [ ]:
df.info()

**Приведем название столбца 'cityCenters_nearest' к "змеиному" регистру, переименовав в 'distance_to_center'**

In [ ]:
df = df.rename(columns={'cityCenters_nearest':'distance_to_center'})

**Построим гистограмму для всех числовых столбцов таблицы на одном графике**

In [ ]:
df.hist(figsize=(15, 20), color='dodgerblue');

### Описание данных

**Для исследования нам предоставлены данные о продажах квартир в Санкт-Петербурге за 6 лет — с 2014 по 2019 годы. Данные содержат 23699 строк в 22 столбцах:**
- airports_nearest — расстояние до ближайшего аэропорта в метрах (м)

- balcony — число балконов

- ceiling_height — высота потолков (м)

- cityCenters_nearest — расстояние до центра города (м)

- days_exposition — сколько дней было размещено объявление (от публикации до снятия)

- first_day_exposition — дата публикации

- floor — этаж

- floors_total — всего этажей в доме

- is_apartment — апартаменты (булев тип)

- kitchen_area — площадь кухни в квадратных метрах (м²)

- last_price — цена на момент снятия с публикации

- living_area — жилая площадь в квадратных метрах (м²)

- locality_name — название населённого пункта

- open_plan — свободная планировка (булев тип)

- parks_around3000 — число парков в радиусе 3 км

- parks_nearest — расстояние до ближайшего парка (м)

- ponds_around3000 — число водоёмов в радиусе 3 км

- ponds_nearest — расстояние до ближайшего водоёма (м)

- rooms — число комнат

- studio — квартира-студия (булев тип)

- total_area — общая площадь квартиры в квадратных метрах (м²)

- total_images — число фотографий квартиры в объявлении

**В 14 столбцах есть пропуски. По ходу работы над проектом мы более подробно познакомимся с данными, рассмотрим каждый столблец, в котором есть пропущенные значения: часть пропусковмы заполним, часть оставим без изменений. Также обработаем, где возможно, аномальные значения. Пропуски в данных могут появиться вследствие человеческого или технического фактора. Например, при создании объявления о продаже квартиры пользователь оставил поле "апартаменты" или "количество балконов" пустым, так как квартира не является апартаментами или не имеет балконов. Также возможны ошибки или частичная утрата данных при выгрузке. Большое количество пропусков в столбцах, заполняемых автоматически на основе данных геосервисов, могут появиться вследвие некорректной работы последних.**

## Предобработка данных

### Удаление пропусков

**Выведем количество пропущенных значений для каждого столбца**

In [ ]:
df.isna().sum()

**Пропуски есть в 14 столбцах:**

**По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Пропуски в следующих столбцах могли появиться в связи с тем, что пользователи не указали в объявлении соответствующую информацию. 
Пропуски в столбце с количеством балконов, вероятно, означают, что балкона в квартире нет. Заменим пропущенные значения на 0. Далее заполним пропуски в следующих столбцах:** 

- balcony                 11519

- is_apartment            20924

- ceiling_height           9195

- living_area              1903

- kitchen_area             2278

**Пропуски в столбце floors_total оставим без изменений, их всего 86, погоды они не сыграют.**

**Пропуски в следующих столбцах относятся в другому виду данных — полученных автоматически на основе картографических данных. Такие данные как расстояние до ближайшего аэропорта, центра и других объектов, количество парков и водоёмов — автоматически получены из геосервисов. Пропуски в соответствующих объявлениях могут быть связаны с большой удаленностью от объектов, отсутствием данных или ошибками при обработке и выгрузке данных. Оставим пропуски в следующих столбцах без изменений:**

- locality_name              49

- distance_to_center      5519

- parks_around3000         5518

- parks_nearest           15620

- ponds_around3000         5518

- ponds_nearest           14589

- days_exposition          3181

- airports_nearest         5542


**Посмотрим, какие значения есть в столбце с количеством балконов**

In [ ]:
df['balcony'].value_counts()

**Заполним пропуски нулями и изменим тип данных столбца на int64**

In [ ]:
df['balcony'] = df['balcony'].fillna(0) 
df['balcony'] = df['balcony'].astype('int64')

**Изучим информацию о столбце is_apartment**

In [ ]:
df['is_apartment'].value_counts()

**Видим, что апартаментами являются только 50 объектов на продажу. Количество незаполненных строк в столбце is_apartment более 20000. Логично предположить, что такое большое количество объектов не может быть апартаментами. Заполним пропуски значением False.**

In [ ]:
df['is_apartment'] = df['is_apartment'].fillna(False)

**Посмотрим уникальные значения в столбце ceiling_height** (количество пропусков — 9195)

In [ ]:
np.sort(df['ceiling_height'].unique())

**В столбце наблюдаем выбросы: значения высоты потолков 100 метров, 20-32 метра, 14, 10, а также значения менее 2 метров. Вероятно, при заполнении или выгрузке данных были допущены ошибки, вследствие которых появились значения более 20 метров. Эти значения исправим путем деления на 10. Строки, в которых указана высота потолков больше 7 метров и строки, в которых потолки ниже 2 метров, удалим.** 

In [ ]:
df.loc[df['ceiling_height'] >= 20, 'ceiling_height'] = df.loc[df['ceiling_height'] >= 20, 'ceiling_height'] / 10

In [ ]:
df = df[(df['ceiling_height'] < 1) | (df['ceiling_height'] >= 2) | df['ceiling_height'].isna()]

In [ ]:
df = df[(df['ceiling_height'] < 7) | df['ceiling_height'].isna()]

**В центре Санкт-Петербурга сохранилось много зданий дореволюционной постройки, а также так называемые "сталинки". Дома современной постройки, находящиеся в центре города или недалеко от него, будут относиться к премиальному классу жилья, поэтому потолки там тоже высокие. Рассмотрим корреляцию отдаленности квартиры от центра и высоты потолков.**

In [ ]:
df['ceiling_height'].corr(df['distance_to_center'])
df.plot(x='distance_to_center', y='ceiling_height', kind='scatter', 
        title='Зависимость высоты потолков от удаленности объекта от центра города', color='dodgerblue', alpha=0.5); 

**Наблюдаем уменьшение высоты потолков по мере удаления от центра города. С помощью функции категоризируем объекты по расстоянию до центра и заполним пропуски. На графике видно, что имеется небольшое количество выбросов (значения 4,5-6 метров), поэтому для заполнения пропусков используем медиану, так как она более устойчива к аномальным значениям.**

**Выделим категории следующим образом:**

- 0-2км от центра — 'самый центр';
- 2-5км от центра — 'центр';
- 5-17км от центра — 'в пределах города';
- более 17км от центра — 'область'.

In [ ]:
def categorize_distance(distance):
    try:
        if distance < 2000:
            return 'самый центр'
        if distance < 5000:
            return 'центр'
        if distance <= 17000:
            return 'в пределах города'
        if distance > 17000:
            return 'область'
    except:
        pass        

In [ ]:
df['center_remoteness_cat'] = df['distance_to_center'].apply(categorize_distance) # применим функцию

**Посмотрим, что получилось:**

In [ ]:
df[['locality_name','distance_to_center','center_remoteness_cat']].head(10)

**В функции мы прописали except-pass, поэтому объектам с пропущенным расстоянием до центра категория не присвоится — некоторое количество пропусков в столбце "высота потолка" все же останется. Заполним пропуски в столбце ceiling_height медианным значением по категории отдаленности от центра.**

In [ ]:
for c in df['center_remoteness_cat'].unique():
    df.loc[(df['center_remoteness_cat'] == c) & (df['ceiling_height'].isna()), 'ceiling_height'] = \
    df.loc[(df['center_remoteness_cat'] == c), 'ceiling_height'].median()


**Посмотрим, сколько осталось пропусков:**

In [ ]:
df['ceiling_height'].isna().sum()

**И распределение высоты потолков по удаленности от центра:**

In [ ]:
df.groupby('center_remoteness_cat')['ceiling_height'].mean().sort_values(ascending=False)

**Уже лучше. Оставшиеся пропуски оставим без изменений. Посмотрим столбец "площадь кухни":**

- kitchen_area 2278

In [ ]:
np.sort(df['kitchen_area'].unique())

**Посмотрим, указана ли площадь кухни в объявлениях о продаже квартир-студий:**

In [ ]:
studio = df[df['studio'] == True]
print('Количество пропусков в объявлениях о продаже студий:', studio['kitchen_area'].isnull().sum())

**Заполним нулями**

In [ ]:
df.loc[(df['studio'] == True) & (df['kitchen_area'].isna()), 'kitchen_area'] = 0

**Посмотрим корреляцию и построим матрицу рассеяния, чтобы проследить взаимосвязь площади кухни, общей площади и жилой площади объекта**

In [ ]:
pd.plotting.scatter_matrix(df[['kitchen_area', 'living_area', 'total_area']], figsize=(10,10), color='dodgerblue', alpha=0.5)
df[['kitchen_area', 'living_area', 'total_area']].corr()

**Видим на графике, что с ростом общей и жилой площади квартиры также увеличивается и площадь кухни. Корреляция площади кухни и жилой площади составляет 0,4, а корреляция площади кухни и общей площади составляет 0,6. Используем эту зависимость для заполнения пропусков в столбце kitchen_area: как и в прошлый раз, категоризируем объекты по общей площади и заполним пропуски в столбце kitchen_area средним значением. 
Также на матрице видим высокую корреляцию общей и жилой площади — ее значение составляет 0,9. Используем это для заполнения пропусков в столбце living_area.**

In [ ]:
df['total_area'].plot(color='dodgerblue');

In [ ]:
df['total_area'].hist(range=(10,200), color='dodgerblue');

**Видим на графиках, что площади квартир варьируются от 10 до 800 квадратных метров. Для удобства разделим объекты на категории:**
- до 25 квадратных метров;
- от 25 до 50 квадратных метров;
- от 50 до 100 квадратных метров;
- от 100 до 200 квадратных метров;
- больше 200 квадратных метров. 

In [ ]:
def categorize_total_area(total_area):
    try:
        if total_area < 50:
            return 'до 50 м.кв.'
        if total_area < 100:
            return 'от 50 до 100 м.кв.'
        if total_area <= 200:
            return 'от 100 до 200 м.кв.'
        if total_area > 200:
            return 'больше 200 м.кв.'
    except:
        pass   

**Добавим столбец total_area_cat, применив к данным функцию categorize_total_area**

In [ ]:
df['total_area_cat'] = df['total_area'].apply(categorize_total_area)

**Посмотрим распределение и среднюю площадь по каждой категории.**

In [ ]:
df['total_area_cat'].value_counts().sort_values(ascending=False) # сколько квартир в каждой категории

In [ ]:
df.groupby('total_area_cat')['total_area'].mean().sort_values(ascending=False) # средняя площадь квартиры в категории

**Заполняем пропуски в столбце "площадь кухни" средним значением нового столбца**

In [ ]:
for a in df['total_area_cat'].unique():
    df.loc[(df['total_area_cat'] == a) & (df['kitchen_area'].isna()), 'kitchen_area'] = \
    df.loc[(df['total_area_cat'] == a), 'kitchen_area'].mean()

**Посмотрим среднюю площадь кухни по каждой категории:**

In [ ]:
df.groupby('total_area_cat')['kitchen_area'].mean().sort_values(ascending=False)

**Аналогичным образом заполним пропуски в столбце living_area — средним значением по категории**

In [ ]:
for liv in df['total_area_cat'].unique():
    df.loc[(df['total_area_cat'] == liv) & (df['living_area'].isna()), 'living_area'] = \
    df.loc[(df['total_area_cat'] == liv), 'living_area'].mean()

**Посмотрим распределение средней жилой площади по каждой категории:**

In [ ]:
df.groupby('total_area_cat')['living_area'].mean().sort_values(ascending=False)

### Изменение типов данных

**Изменим тип данных в следующих столбцах:**

- floors_total, parks_around3000, ponds_around3000, days_exposition с float64 на int64

- first_day_exposition с object64 на datetime64 для удобства дальнейшей работы с данными этого столбца

- living_area и kitchen_area тип данных оставим без изменений, но преобразуем до значения с 1 знаком после запятой

- last_price итоговую цену переведем в тысячи для более удобного восприятия

- airports_nearest и distance_to_center переведем в километры, приведем к типу данных int64 


In [ ]:
df[['airports_nearest', 'distance_to_center', 'last_price']] = df[['airports_nearest', 'distance_to_center', 'last_price']] / 1000

In [ ]:
to_int_list = ['last_price', 'floors_total', 'parks_around3000', 'ponds_around3000', 'days_exposition', 'airports_nearest', 'distance_to_center']
  
try:
    for column in to_int_list:
        df[column] = df[df[column].notnull()][column].astype('int64')
except:
    pass

In [ ]:
df[['living_area', 'total_area', 'kitchen_area']] = df[['living_area', 'total_area', 'kitchen_area']].round(1)

In [ ]:
df['first_day_exposition'] = pd.to_datetime(df['first_day_exposition'])

In [ ]:
df.head(10)

### Устранение неявных дубликатов

**Проверим наличие строк-дубликатов:**

In [ ]:
df.duplicated().sum()

Явных дубликатов нет, едем дальше.

**Просмотрим уникальные значения столбца locality_name для поиска и устранения неявных дубликатов:**

In [ ]:
df['locality_name'].unique()

In [ ]:
df['locality_name'] = df['locality_name'].str.replace('ё', 'е')

df['locality_name'] = df['locality_name'].apply(lambda x: re.sub(r'^(деревня|село|поселок городского типа|городской поселок|коттеджный поселок|\
                                                                 поселок станции|поселок при железнодорожной станции|поселок|садовое товарищество|\
                                                                 садоводческое некоммерческое товарищество)\s', '', str(x)))

# Проверка результата
df['locality_name'].unique()


### Добавление в таблицу новых столбцов


**Цена одного квадратного метра**

In [ ]:
df.loc[:, 'meter2_price'] = round((df['last_price'] / df['total_area']), 2)

**День недели публикации объявления (0 — понедельник, 1 — вторник и так далее)**

In [ ]:
df.loc[:, 'day_of_exposition'] = df['first_day_exposition'].dt.weekday

**Месяц публикации объявления**

In [ ]:
df.loc[:, 'month_of_exposition'] = df['first_day_exposition'].dt.month

**Год публикации объявления**

In [ ]:
df.loc[:, 'year_of_exposition'] = df['first_day_exposition'].dt.year
np.sort(df['year_of_exposition'].unique())

**Тип этажа квартиры (значения — «первый», «последний», «другой»)**

In [ ]:
np.sort(df['floors_total'].unique())

60 этажей — неожиданный результат. Пока оставим так для добавления столбца, позже обработаем аномальные значения. Создадим функцию:

In [ ]:
try:
    def categorize_floor(row):
        if row['floor'] == 1:
            return 'первый'
        elif row['floor'] == row['floors_total']:
            return 'последний'
        else:
            return 'другой'
except: 
    pass

In [ ]:
df['floor_type'] = df.apply(categorize_floor, axis=1) # применяем функцию к датафрейму

**Посмотрим на результат:**

In [ ]:
df[['meter2_price', 'day_of_exposition', 'month_of_exposition', 'year_of_exposition', 'floor_type']].head()

### Вывод

**Как мы увидели, данные достаточно загрязнены:**
- ошибками (высота потолков записана в целочисленном формате вместо вещественного, что привело к появлению квартир с потолками высотой 20-30 метров);

- пропусками, как в данных, предоставляемых пользователями, так и в данных, заполняемых автоматически;

- неявными дубликатами в столбце 'locality_name': имеются повторяющиеся названия населенных пунктов: «поселок Рябово» и «поселок городского типа Рябово», «поселок Тельмана» и «посёлок Тельмана» и так далее;

- несоответствием форматов.

**В ходе предварительной обработки были выполнены следующие действия:**
1. Обработаны пропуски в столбцах

- balcony заменены на 0

- is_apartment заменены на False

- ceiling_height произведена категоризация объектов по удаленности от центра города, пропуски заполнены медианным значением по категории

- living_area и kitchen_area заполнены средним значением по категории "общая площадь" в связи с высокой корреляцией данных показателей

2. Изменили тип и представление данных в следующих столбцах:

- floors_total, parks_around3000, ponds_around3000, days_exposition с float64 на int64

- first_day_exposition с object64 на datetime64 для удобства дальнейшей работы с данными этого столбца

- living_area и kitchen_area преобразовали до значения с 1 знаком после запятой

- last_price итоговую цену перевели в тысячи для более удобного восприятия

- airports_nearest и distance_to_center перевели в километры, привели к типу данных int64 

3. Добавили в талицу новые столбцы:

- meter2_price средняя цена за квадратный метр

- day_of_exposition	день недели, когда было размещено объявление

- month_of_exposition месяц публикации объявления

- year_of_exposition год публикации объявления

- floor_type тип этажа квартиры (первый, последний, другой)

**Теперь данные готовы для дальнейшего анализа.**

## Исследовательский анализ данных

**В этом блоке изучим перечисленные ниже параметры объектов. По возможности удалим аномалии. 
Список параметров:**
- общая площадь;
- жилая площадь;
- площадь кухни;
- цена объекта;
- количество комнат;
- высота потолков;
- тип этажа квартиры («первый», «последний», «другой»);
- общее количество этажей в доме;
- расстояние до центра города в метрах;
- расстояние до ближайшего парка**

**Изучим, как быстро продавались квартиры (столбец days_exposition). Этот параметр показывает, сколько дней было размещено каждое объявление.
Посмотрим, сколько времени обычно занимает продажа: какие продажи можно считать быстрыми, а какие — необычно долгими?
Определим факторы, которые больше всего влияют на общую (полную) стоимость объекта.
Изучим, зависит ли цена от:**
- общей площади;
- жилой площади;
- площади кухни;
- количества комнат;
- этажа, на котором расположена квартира (первый, последний, другой);
- даты размещения (день недели, месяц, год).**

**Посчитаем среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений. Выделим населённые пункты с самой высокой и низкой стоимостью квадратного метра.**

**Ранее мы посчитали расстояние до центра в километрах. Теперь вычислим среднюю стоимость квартир на разном удалении от центра Санкт-Петербурга. Опишем, как стоимость объектов зависит от расстояния до центра города.**

### Изучаем данные на предмет наличия аномальных значений

**Изучим данные столбцов 'total_area', 'living_area', 'kitchen_area', 'last_price' и построим диаграмму размаха.**

In [ ]:
df[['total_area', 'living_area', 'kitchen_area', 'last_price']].describe()

In [ ]:
rows = 2
cols = 2
fig, axes = plt.subplots(rows, cols, figsize=(50,25))
column_list = ['total_area', 'living_area', 'kitchen_area', 'last_price']
ylims = [300, 200, 45, 20000]
count = 0
for r in range(rows):
    for c in range(cols):
        ax = df.boxplot(column=column_list[count], ax=axes[r,c], fontsize=40)
        ax.set_ylim([0, ylims[count]])
        count += 1

**Значения распределяются следующим образом:**
- 'total_area' — в размах "усов" падает метраж от 12 до 120 м.кв. Имеется большое количество выбросов — квартиры общей площадью 300-600 кв.метров и даже больше. Например, максимальное значение, полученное при вызове метода decsribe выдало такой результат: общая площадь 900.000000, жилая площадь 409.700000, площадь кухни 112.000000, стоимость	763000.000000. Посмотрим, какое в данных количество квартир общей площадью более 300 м.кв.
- 'living_area' — нормальные значения находятся в диапазоне от 2 до 75 м.кв. 
- 'kitchen_area' — нормальные значения в диапазоне от 1 до 18 м.кв. Также встречаются значения 0 — это квартиры-студии.
- 'last_price' — нормальные значения находятся в диапазоне от 0 до 12 млн.ед.

In [ ]:
df['total_area'].hist(figsize=(10,6), range=(300,900), bins=50, color='dodgerblue');

**На гистограмме видно, что в представленной выборке имеется небольшое количество квартир общей площадью 300-400 м.кв. (около 40), 400-500 м.кв (примерно 15 таких квартир), 4 квартиры площадью более 600 м.кв. Аномалии могут существенно влиять на среднее значение и стандартное отклонение в наборе данных, что может привести к неверным прогнозам или выводам. Удалим строки с квартирами общей площадью более 300 м.кв., чтобы избежать искажения результатов анализа. Посмотрим на выбросы в столбце last_price.**

In [ ]:
df['last_price'].hist(figsize=(10,6), range=(20000,77000), bins=50, color='dodgerblue');

**В данных имеются квартиры стоимостью 20-30 млн.ед. (примерно 300 таких), 30-40 млн.ед. — порядка 150 квартир и дороже 40 млн. — 182 квартиры. Для более точного анализа оставим объявления о квартирах стоимостью до 20 млн., и еще удалим строку, в которой  указана стоимость 12 тысяч.**

**Дальше посмотрим столбцы 'rooms' и'ceiling_height'. Ранее мы обработали аномальные значения в столбце с высотой потолка.**

In [ ]:
df[['rooms', 'ceiling_height']].describe()

In [ ]:
plt.ylim(0, 10)
df[['rooms', 'ceiling_height']].boxplot();

**Наблюдаем:**
- 'rooms' нормальные значения от 0 до 6 комнат, максимальное значение 19. 
- 'ceiling_height' — нормальные значения в диапазоне от 2,6 до 2,75 м. Встречаются значения меньше (2-2,5 метра) и больше — до 6 м.

In [ ]:
len(df[df['rooms'] > 6]) 

**Общее количество квартир, в которых больше 6 комнат — 90. Удалим их, чтобы не искажали результаты анализа.**

In [ ]:
df = df[(df['last_price'] > 100) & (df['last_price'] < 20000) & (df['total_area'] < 300) & (df['rooms'] < 6)]        

Посмотрим отношение нового размера к старому, чтобы узнать, как изменился размер датафрейма.

In [ ]:
new_size = df.shape[0]
round((new_size / old_size)*100, 2)

В пределах допустимых значений. Можно продолжать.

**Посмотрим распределение типов этажа квартиры («первый», «последний», «другой»)**

In [ ]:
floor_count = df['floor_type'].value_counts()
plt.figure(figsize=(10, 6))
plt.pie(floor_count, labels=floor_count.index, autopct='%1.1f%%')
plt.title('Распределение типов этажей')
plt.show()

**12,5 % квартир находятся на 1 этаже, 13,8 % — на последнем. Большая часть объектов (73,6 %) располагается на других этажах.**

**Посмотрим распределение общего количества этажей в доме**

In [ ]:
df['floors_total'].describe()

In [ ]:
df['floors_total'].hist(range=(1,37), bins=37, color='dodgerblue');

**Большинство квартир находятся в пятиэтажных домах, на втором месте — девятиэтажки. Дальше идут 16-ти и 25-этажные дома. Самый высокий дом в Санкт-Петербурге в ЖК Князь Александр Невский — 37 этажей. удалим строки с количеством этажей больше 37.**

In [ ]:
df = df[(df['floors_total'] < 37) | df['floors_total'].isna()]

**Расстояние до центра города в метрах**

In [ ]:
df[['distance_to_center', 'parks_nearest']].describe()

In [ ]:
df['distance_to_center'].hist(bins=50, color='dodgerblue');

**Значения расстояния до центра в диапазоне от 180 м до 66 км. Больше всего квартир находятся на расстоянии от 10 до 20 км от центра Санкт-Петербурга.**

In [ ]:
df[df['distance_to_center'] < 0.2]

**Вполне правдоподобно, можно оставить. Дальше посмотрим расстояние до ближайшего парка**

In [ ]:
df['parks_nearest'].hist(color='dodgerblue');

**Если в радиусе 3000 метров от квартиры есть парк, то чаще всего он в радиусе 1000 метров. Похоже, что данные, заполняемые автоматически с помощью геосервиса, заполнены корректно (без выбросов), за исключением пропусков.**

### Анализ времени продажи квартир (столбец days_exposition)

**Построим гистограмму, чтобы посмотреть за сколько дней продавались квартиры.**

In [ ]:
df['days_exposition'].hist(bins=50, color='dodgerblue');

**Более 4000 квартир были проданы меньше, чем за 100 дней. После отметки 100 идет резкий спуск и хвост гистограммы. Посмотрим поближе на пиковые значения.**

In [ ]:
df['days_exposition'].hist(range=(0,200), bins=50, color='dodgerblue');

**На гистограмме видим пики в районе 45 и 60 дней, небольшой пик в районе 85; устойчивое равномерное снижение числа продаж с 5 до 200 дня с момента размещения объявления. По правилам Яндекс.Недвижимости объявления автоматически снимаются с публикции как раз на 45й и 60й день, в зависимости от типа. Посмотрим среднее и медиану.**

In [ ]:
df['days_exposition'].mean()

In [ ]:
df['days_exposition'].median()

In [ ]:
df['days_exposition'].describe()

**Среднее время продажи составляет примерно 181 день. Однако стандартное отклонение достаточно велико (220 дней), что указывает на значительное разнообразие в данных.**

Быстрые продажи: Если мы рассмотрим первый квартиль, то увидим, что 25% всех продаж были совершены за 44 дней или меньше.

Средние продажи: Медианное значение составляет 93 дней. Это означает, что половина всех продаж была совершена за 93 дней или меньше.

Необычно долгие продажи: Если мы рассмотрим третий квартиль, то увидим, что 75% всех продаж были совершены за 226 дней или меньше. Продажи, которые занимают больше времени, можно считать необычно долгими.

Максимальное значение в данных составляет 1580 дней, что является очень долгим временем для продажи. Однако это максимальное значение, и такие случаи скорее исключение, чем правило.

### Изучение зависимости цены объекта от различных факторов. 

**Чтобы узнать, зависит ли цена от: общей площади; жилой площади; площади кухни; количества комнат; этажа, на котором расположена квартира (первый, последний, другой); даты размещения (день недели, месяц, год), построим матрицу рассеяния.**

In [ ]:
columns = ['last_price', 'total_area', 'living_area', 'kitchen_area', 'rooms']
df_matrix = df[columns]
pd.plotting.scatter_matrix(df_matrix, figsize=(10,10), color='dodgerblue');

In [ ]:
df[['last_price', 'total_area', 'living_area', 'kitchen_area', 'rooms']].corr()

**Проанализируем взаимосвязи:**
- total_area: Коэффициент корреляции составляет 0.768, что указывает на сильную положительную взаимосвязь. Это означает, что с увеличением общей площади, цена на недвижимость также, как правило, увеличивается.
- living_area: Коэффициент корреляции составляет 0.654, что также указывает на положительную взаимосвязь, хотя и не такую сильную, как с total_area. Это означает, что с увеличением жилой площади, цена на недвижимость обычно также увеличивается.
- kitchen_area: Коэффициент корреляции составляет 0.571, что указывает на умеренную положительную взаимосвязь. Это означает, что с увеличением площади кухни, цена на недвижимость обычно также увеличивается, хотя эта связь не такая сильная, как с total_area или living_area.
- rooms: Коэффициент корреляции составляет 0.491, что указывает на слабую положительную взаимосвязь. Это означает, что с увеличением количества комнат, цена на недвижимость обычно также увеличивается, но эта связь слабее, чем с другими признаками.

**Изучим зависимость стоимости квартиры от дня, месяца и года публикации объявления.**

In [ ]:
# columns = ['last_price', 'day_of_exposition', 'month_of_exposition', 'year_of_exposition']

mean_price_per_day = df.groupby('day_of_exposition')['last_price'].mean()
mean_price_per_month = df.groupby('month_of_exposition')['last_price'].mean()
mean_price_per_year = df.groupby('year_of_exposition')['last_price'].mean()

fig, ax = plt.subplots(3, 1, figsize=(10, 15))
mean_price_per_day.plot(kind='bar', ax=ax[0])
ax[0].set_title('Зависимость средней цены от дня недели')
ax[0].set_xlabel('День недели')
ax[0].set_ylabel('Средняя цена')

# График зависимости средней цены от месяца
mean_price_per_month.plot(kind='bar', ax=ax[1])
ax[1].set_title('Зависимость средней цены от месяца')
ax[1].set_xlabel('Месяц')
ax[1].set_ylabel('Средняя цена')

# График зависимости средней цены от года
mean_price_per_year.plot(kind='bar', ax=ax[2])
ax[2].set_title('Зависимость средней цены от года')
ax[2].set_xlabel('Год')
ax[2].set_ylabel('Средняя цена')

plt.tight_layout()
plt.show()

**Итак:**
- стоимость квартиры от дня недели публикации практически никак не зависит;
- зависимость цены от месяца публикации: цены на квартиры, чьи объявления о продаже были опубликованы в апреле и сентябре, немного выше, чем по остальным месяцам;
- зависимость цены от года публикации — в 2014 году цены на квартиры были самые высокие, в 2015-2017 годах можно наблюдать плавное снижение цен, в 2018 году цены на квартиры остаются примерно на том же уровне, а в 2019 году наблюдаем рост цен на недвижимость.


**Связь этажа и стоимости квартиры**

In [ ]:
plt.figure(figsize=(6,4))
sns.barplot(x='floor_type', y='last_price', data=df)

plt.title('Взаимосвязь между last_price и floor_type')
plt.show()

**Наблюдаем, что квартиры на последнем этаже самые дорогие, немного  меньше стоят квартиры на остальных этажах и самые низкие цены на квартиры, находящиеся на первом этаже.**

**Посчитаем среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений — построим сводную таблицу с количеством объявлений и средней ценой квадратного метра для этих населенных пунктов.**

In [ ]:
grouped = df.groupby('locality_name').agg({'meter2_price': 'mean', 'locality_name': 'count'})
grouped.columns = ['Средняя стоимость квадратного метра', 'Количество объявлений']
top_10_localities = grouped.sort_values('Количество объявлений', ascending=False).head(10)
top_10_localities.sort_values('Средняя стоимость квадратного метра', ascending=False)

In [ ]:
plt.figure(figsize=(10,6))
top_10_localities['Средняя стоимость квадратного метра'].sort_values(ascending=False).plot(kind='bar');

**Самая высокая стоимость квадратного метра в Санкт-Петербурге и Пушкине. Самая низкая цена за квадратный метр в Выборге и Всеволожске.**

Ранее мы посчитали расстояние до центра в километрах. Теперь выделим квартиры в Санкт-Петербурге с помощью столбца locality_name и вычислим их среднюю стоимость на разном удалении от центра. 

In [ ]:
center_price = df[df['locality_name'] == 'Санкт-Петербург']
center_price.loc[:, 'distance_to_center_km'] = np.round(center_price['distance_to_center'])
average_center_price = center_price.groupby('distance_to_center_km')['last_price'].mean().round(1)
average_center_price.columns = ['Расстояние до центра, км', 'Средняя стоимость']
average_center_price.head(10)

**Посмотрим на графике, как изменяется средняя стоимость квартир по мере удаления от центра Санкт-Петербурга:**

In [ ]:
average_center_price_plt = average_center_price.head(10)
plt.title('Зависимость средней стоимости квартиры от удаленности от центра Санкт-Петербурга')
plt.xlabel('расстояние до центра, км')
plt.ylabel('средняя стоимость')
plt.xticks(np.arange(0, 10, step=1)) # изменим шаг на оси x 
plt.plot(average_center_price_plt);




**Из данных видно, что стоимость квартиры в Санкт-Петербурге зависит от расстояния до центра города:**

- квартиры, расположенные в центре города (0 км), имеют самую высокую среднюю стоимость, равную 31449.1.
- с увеличением расстояния до центра города средняя стоимость квартир уменьшается. Например, квартиры, расположенные на расстоянии 1 км от центра, имеют среднюю стоимость 10098.5, а квартиры на расстоянии 2 км -  9447.8.
- однако есть некоторые исключения из этого общего тренда. Например, средняя стоимость квартир, расположенных на расстоянии 4 км от центра (9529.4), выше, чем стоимость квартир на расстоянии 3 км (8877.0). Вероятно, это Василеостровский район дает такой показатель.
- квартиры, расположенные на расстоянии 9 км от центра, имеют самую низкую среднюю стоимость, равную 6018.8.

### Вывод

В этом блоке изучили перечисленные ниже параметры объектов. 
Значения распределяются следующим образом:

- 'total_area' — в размах "усов" падает метраж от 12 до 120 м.кв. Удалили строки со значениями выше 300 м.кв.
- 'living_area' — нормальные значения находятся в диапазоне от 2 до 75 м.кв.
- 'kitchen_area' — нормальные значения в диапазоне от 1 до 18 м.кв. Также встречаются значения 0 — это квартиры-студии.
- 'last_price' — нормальные значения находятся в диапазоне от 0 до 12 млн.ед. Квартиры стоимостью больше 20 млн.ед. удалили из выборки для более точного анализа. 
- 'rooms' нормальные значения от 0 до 6 комнат, максимальное значение 19. Удалили строки со значением выше 6
- 'ceiling_height' — нормальные значения в диапазоне от 2,6 до 2,75 м. 
- 'floor' — 12,5 % квартир находятся на 1 этаже, 13,8 % — на последнем. Большая часть объектов (73,6 %) располагается на других этажах.
- 'floors_total' — большинство квартир находятся в пятиэтажных домах, на втором месте — девятиэтажки. Дальше идут 16-ти и 25-этажные дома. Удалили аномальные значения в столбце.
- 'distance_to_center' — значения расстояния до центра в диапазоне от 180 м до 66 км. Больше всего квартир находятся на расстоянии от 10 до 20 км от центра Санкт-Петербурга.
- parks_nearest — если в радиусе 3000 метров от квартиры есть парк, то чаще всего он в радиусе 1000 метров. 

Изучили, как быстро продавались квартиры (столбец days_exposition). Среднее время продажи составляет примерно 181 день. Однако стандартное отклонение достаточно велико (220 дней), что указывает на значительное разнообразие в данных. Быстрые продажи: 25% всех продаж были совершены за 44 дней или меньше. Средние продажи: половина всех продаж была совершена за 93 дней или меньше. Необычно долгие продажи: 75% всех продаж были совершены за 226 дней или меньше. Продажи, которые занимают больше времени, можно считать необычно долгими. Максимальное значение в данных составляет 1580 дней, что является очень долгим временем для продажи. Однако это максимальное значение, и такие случаи скорее исключение, чем правило. 

Узнали, зависит ли цена от: общей площади; жилой площади; площади кухни; количества комнат; этажа, на котором расположена квартира (первый, последний, другой); даты размещения (день недели, месяц, год).

- total_area: Коэффициент корреляции составляет 0.768, что указывает на сильную положительную взаимосвязь. Это означает, что с увеличением общей площади, цена на недвижимость также, как правило, увеличивается.
- living_area: Коэффициент корреляции составляет 0.654, что также указывает на положительную взаимосвязь, хотя и не такую сильную, как с total_area. Это означает, что с увеличением жилой площади, цена на недвижимость обычно также увеличивается.
- kitchen_area: Коэффициент корреляции составляет 0.571, что указывает на умеренную положительную взаимосвязь. Это означает, что с увеличением площади кухни, цена на недвижимость обычно также увеличивается, хотя эта связь не такая сильная, как с total_area или living_area.
- rooms: Коэффициент корреляции составляет 0.491, что указывает на слабую положительную взаимосвязь. Это означает, что с увеличением количества комнат, цена на недвижимость обычно также увеличивается, но эта связь слабее, чем с другими признаками.

Изучили зависимость стоимости квартиры от дня, месяца и года публикации объявлени.

- стоимость квартиры от дня недели публикации практически никак не зависит;
- зависимость цены от месяца публикации: цены на квартиры, чьи объявления о продаже были опубликованы в апреле и сентябре, немного выше, чем по остальным месяцам;
- зависимость цены от года публикации — в 2014 году цены на квартиры были самые высокие, в 2015-2017 годах можно наблюдать плавное снижение цен, в 2018 году цены на квартиры остаются примерно на том же уровне, а в 2019 году наблюдаем рост цен на недвижимость.

Наблюдаем, что квартиры на последнем этаже самые дорогие, немного меньше стоят квартиры на остальных этажах и самые низкие цены на квартиры, находящиеся на первом этаже. 

Из данных видно, что стоимость квартиры в Санкт-Петербурге зависит от расстояния до центра города:

- квартиры, расположенные в центре города (0 км), имеют самую высокую среднюю стоимость, равную 31449.1.
- с увеличением расстояния до центра города средняя стоимость квартир уменьшается. Например, квартиры, расположенные на расстоянии 1 км от центра, имеют среднюю стоимость 10098.5, а квартиры на расстоянии 2 км -  9447.8.
- однако есть некоторые исключения из этого общего тренда. Например, средняя стоимость квартир, расположенных на расстоянии 4 км от центра (9529.4), выше, чем стоимость квартир на расстоянии 3 км (8877.0). Вероятно, это Василеостровский район дает такой показатель.
- квартиры, расположенные на расстоянии 9 км от центра, имеют самую низкую среднюю стоимость, равную 6018.8.

### Общий вывод

**Предварительная обработка данных**

Для исследования предоставлены данные о продажах квартир в Санкт-Петербурге за 6 лет — с 2014 по 2019 годы. Данные содержат 23699 строк в 22 столбцах:

- airports_nearest — расстояние до ближайшего аэропорта в метрах (м)

- balcony — число балконов

- ceiling_height — высота потолков (м)

- cityCenters_nearest — расстояние до центра города (м)

- days_exposition — сколько дней было размещено объявление (от публикации до снятия)

- first_day_exposition — дата публикации

- floor — этаж

- floors_total — всего этажей в доме

- is_apartment — апартаменты (булев тип)

- kitchen_area — площадь кухни в квадратных метрах (м²)

- last_price — цена на момент снятия с публикации

- living_area — жилая площадь в квадратных метрах (м²)

- locality_name — название населённого пункта

- open_plan — свободная планировка (булев тип)

- parks_around3000 — число парков в радиусе 3 км

- parks_nearest — расстояние до ближайшего парка (м)

- ponds_around3000 — число водоёмов в радиусе 3 км

- ponds_nearest — расстояние до ближайшего водоёма (м)

- rooms — число комнат

- studio — квартира-студия (булев тип)

- total_area — общая площадь квартиры в квадратных метрах (м²)

- total_images — число фотографий квартиры в объявлении

В 14 столбцах были обнаружены пропуски. Пропуски в данных могут появиться вследствие человеческого или технического фактора. Например, при создании объявления о продаже квартиры пользователь оставил поле "апартаменты" или "количество балконов" пустым, так как квартира не является апартаментами или не имеет балконов. Также возможны ошибки или частичная утрата данных при выгрузке. Большое количество пропусков в столбцах, заполняемых автоматически на основе данных геосервисов, могут появиться вследвие некорректной работы последних.
Представленные данные достаточно загрязнены:

- ошибками (высота потолков записана в целочисленном формате вместо вещественного, что привело к появлению квартир с потолками высотой 20-30 метров);

- пропусками, как в данных, предоставляемых пользователями, так и в данных, заполняемых автоматически;

- неявными дубликатами в столбце 'locality_name': имеются повторяющиеся названия населенных пунктов: «поселок Рябово» и «поселок городского типа Рябово», «поселок Тельмана» и «посёлок Тельмана» и так далее;

- несоответствием форматов.

В ходе предварительной обработки были выполнены следующие действия:

Обработаны пропуски в столбцах
- balcony заменены на 0

- is_apartment заменены на False

- ceiling_height произведена категоризация объектов по удаленности от центра города, пропуски заполнены медианным значением по категории

- living_area и kitchen_area заполнены средним значением по категории "общая площадь" в связи с высокой корреляцией данных показателей.


Изменили тип и представление данных в следующих столбцах:

- floors_total, parks_around3000, ponds_around3000, days_exposition с float64 на int64

- first_day_exposition с object64 на datetime64 для удобства дальнейшей работы с данными этого столбца

- living_area и kitchen_area преобразовали до значения с 1 знаком после запятой

- last_price итоговую цену перевели в тысячи для более удобного восприятия

- airports_nearest и distance_to_center перевели в километры, привели к типу данных int64


**Проведен исследовательский анализ данных. Параметры объектов распределяются следующим образом:**

- 'total_area' — в размах "усов" падает метраж от 12 до 120 м.кв. Удалили строки со значениями выше 300 м.кв.
- 'living_area' — нормальные значения находятся в диапазоне от 2 до 75 м.кв.
- 'kitchen_area' — нормальные значения в диапазоне от 1 до 18 м.кв. Также встречаются значения 0 — это квартиры-студии.
- 'last_price' — нормальные значения находятся в диапазоне от 0 до 12 млн.ед. Квартиры стоимостью больше 20 млн.ед. удалили из выборки для более точного анализа. 
- 'rooms' нормальные значения от 0 до 6 комнат, максимальное значение 19 
- 'ceiling_height' — нормальные значения в диапазоне от 2,6 до 2,75 м. 
- 'floor' — 12,5 % квартир находятся на 1 этаже, 13,8 % — на последнем. Большая часть объектов (73,6 %) располагается на других этажах.
- 'floors_total' — большинство квартир находятся в пятиэтажных домах, на втором месте — девятиэтажки. Дальше идут 16-ти и 25-этажные дома. Удалили аномальные значения в столбце.
- 'distance_to_center' — значения расстояния до центра в диапазоне от 180 м до 66 км. Больше всего квартир находятся на расстоянии от 10 до 20 км от центра Санкт-Петербурга.
- parks_nearest — если в радиусе 3000 метров от квартиры есть парк, то чаще всего он в радиусе 1000 метров. 

Среднее время продажи составляет примерно 181 день. Однако стандартное отклонение достаточно велико (220 дней), что указывает на значительное разнообразие в данных. Быстрые продажи: 25% всех продаж были совершены за 44 дней или меньше. Средние продажи: половина всех продаж была совершена за 93 дней или меньше. Необычно долгие продажи: 75% всех продаж были совершены за 226 дней или меньше. Продажи, которые занимают больше времени, можно считать необычно долгими. Максимальное значение в данных составляет 1580 дней, что является очень долгим временем для продажи. Однако это максимальное значение, и такие случаи скорее исключение, чем правило. 

Взаимосвязь стоимости квартиры и различных параметров:

- total_area: Коэффициент корреляции составляет 0.768, что указывает на сильную положительную взаимосвязь. Это означает, что с увеличением общей площади, цена на недвижимость также, как правило, увеличивается.
- living_area: Коэффициент корреляции составляет 0.654, что также указывает на положительную взаимосвязь, хотя и не такую сильную, как с total_area. Это означает, что с увеличением жилой площади, цена на недвижимость обычно также увеличивается.
- kitchen_area: Коэффициент корреляции составляет 0.571, что указывает на умеренную положительную взаимосвязь. Это означает, что с увеличением площади кухни, цена на недвижимость обычно также увеличивается, хотя эта связь не такая сильная, как с total_area или living_area.
- rooms: Коэффициент корреляции составляет 0.491, что указывает на слабую положительную взаимосвязь. Это означает, что с увеличением количества комнат, цена на недвижимость обычно также увеличивается, но эта связь слабее, чем с другими признаками.

Изучили зависимость стоимости квартиры от дня, месяца и года публикации объявлени.

- стоимость квартиры от дня недели публикации практически никак не зависит;
- зависимость цены от месяца публикации: цены на квартиры, чьи объявления о продаже были опубликованы в апреле и сентябре, немного выше, чем по остальным месяцам;
- зависимость цены от года публикации — в 2014 году цены на квартиры были самые высокие, в 2015-2017 годах можно наблюдать плавное снижение цен, в 2018 году цены на квартиры остаются примерно на том же уровне, а в 2019 году наблюдаем рост цен на недвижимость.

Квартиры на последнем этаже самые дорогие, немного меньше стоят квартиры на остальных этажах и самые низкие цены на квартиры, находящиеся на первом этаже. 

Из данных видно, что стоимость квартиры в Санкт-Петербурге зависит от расстояния до центра города:

- квартиры, расположенные в центре города (0 км), имеют самую высокую среднюю стоимость, равную 31449.1.
- с увеличением расстояния до центра города средняя стоимость квартир уменьшается. Например, квартиры, расположенные на расстоянии 1 км от центра, имеют среднюю стоимость 10098.5, а квартиры на расстоянии 2 км -  9447.8.
- однако есть некоторые исключения из этого общего тренда. Например, средняя стоимость квартир, расположенных на расстоянии 4 км от центра (9529.4), выше, чем стоимость квартир на расстоянии 3 км (8877.0). Вероятно, это Василеостровский район дает такой показатель.
- квартиры, расположенные на расстоянии 9 км от центра, имеют самую низкую среднюю стоимость, равную 6018.8.

**Рекомендации по сбору данных**

Загрязненные данные усложняют работу и снижают точность анализа, поэтому для более качественного сбора данных рекомендуется:
- столбец locality_name заполнять не вручную, а выбирать наименование населенного пункта из выпадающего списка. Это решит проблему неявных дубликатов в столбце. 
- сделать некоторые поля обязательными для заполнения пользователем, например ceiling_height, либо предусмотреть автоматическое запонение нулями таких полей как kitchen_area и balcony.
- проверить функционирование геосервиса, чтобы в строках, заполняемых автоматически не было пропусков. 

